In [1]:
import chess
import pandas as pd
import numpy as np
import time
import positional_features
from IPython.display import clear_output
from stockfish import Stockfish

DATASET_OLD_PATH = '../data_old/dataset_old.csv'
DATASET_PATH = '../data/dataset.csv'

TEST_DATASET_PATH = '../test/test_dataset.csv'
NEW_TEST_DATASET_PATH = '../test/new_test_dataset.csv'
data = pd.read_csv(DATASET_OLD_PATH, index_col=0)

stockfish = Stockfish(path = 'C:\Program Files\stockfish\stockfish-windows-x86-64-avx2.exe', depth=15)

In [3]:
def get_sf_eval(fen):
    stockfish.set_fen_position(fen)
    eval = stockfish.get_evaluation()
    
    if eval['type'] == 'cp':
        return eval['value'] / 100.0
    else:
        return float('inf') if eval['value'] > 0 else float('-inf')
    
def get_sf_eval_before_and_after(fen, move):
    eval_before = get_sf_eval(fen)
    
    board = chess.Board(fen)
    board.push_uci(move)
    
    eval_after = get_sf_eval(board.fen())
    
    return (eval_before, eval_after)

In [5]:
# adding stockfish evaluation to every position before and after my move & game phase heuristic
size = len(data)
evals = []
game_phases = []

for index, row in data.iterrows():
    if index % 100 == 0:
        clear_output()
        print(f'{index}/{size}')
    
    fen = row['fen']
    move = row['move']
    
    evals.append(get_sf_eval_before_and_after(fen, move))
    game_phases.append(positional_features.game_phase(fen))
    
evals_before, evals_after = zip(*evals)
data['eval_before'] = evals_before
data['eval_after'] = evals_after
data['game_phase'] = game_phases

data.to_csv(DATASET_PATH)

# 105293 positions evaluated in 447 minutes => ~235 positions/minute or ~14000 positions/hour

105200/105293
